# Simple introduction tutorial on PCA

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import scipy.optimize as opt
import sklearn
import matplotlib.pyplot as plt

# Dimesionality reduction

The theme of this tutorial is the concept of dimensionality reduction, which  refers to the idea of representing the information present in set of features with dimensions $N$ by a set of features with dimension $M \ll N$.

First we consider an observation of all the features for a single case which we denote as $\mathbf{x}^{(i)}$ where 
\begin{equation}
\mathbf{x} = \begin{bmatrix} \text{feature 1} & \text{feature 2} & \cdots &\text{feature N}\end{bmatrix}
\end{equation}

We would like to determine a mapping
\begin{equation}
\mathbf{x} \approx \mathbf{A} \mathbf{t}
\end{equation}
where $\mathbf{t} \in \mathbb{R}^{M}$

Thinking of the values of $\mathbf{t}$ as corrdinates in a lower dimensional space we have reduced the dimensionality of the representation of the information in $\mathbf{x}$.

## A *very* simple example

In [ ]:
data = np.random.randn(2,100)
cov_mat = np.array([[2,0.5],[1,0.5]])
data_trans = np.dot(cov_mat, data)
plt.figure("2D-example")
#plt.plot(data[0],data[1],'o')
plt.plot(data_trans[0],data_trans[1],'o')
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")

## Intuitive understanding
* Looking at this figure what do you notice? 
* How would you describe this to someone if they couldn't see the picture?

In [ ]:
plt.figure("2D-example")
plt.quiver(0, 0, cov_mat[0,0], cov_mat[1,0],  angles='xy', scale_units='xy', scale=1, zorder=1e6)

# The idea of PCA 

PCA formalizes this intuitive idea by identifying directions (and thus subspaces) which preserve the most variation when the data vectors are projected onto the subspace.

# Random vectors and the covariance matrix
First, we consider our feature vectors as being generated by some probability distribution
\begin{equation}
\mathbf{x} \sim \mathcal{D}(\mu, \Sigma)
\end{equation}
where $\mu$ is the expected value $E(\mathbf{x}) = \mu$ and $\Sigma$ is the covariance matrix.

The covariance matrix may be defined (for $\mu=0$ WLOG) as
\begin{equation}
\Sigma = E(\mathbf{x}\mathbf{x}^\top)
\end{equation}
(here we treat a single realization as a column vector).

Now we assemble our data matrix by recording each observation of a case (denoted $\mathbf{x}^{(i)}$) as a row:

\begin{equation}
\mathbf{X} = \begin{bmatrix} \mathbf{x}^{(1)} \\
                             \vdots \\
                             \mathbf{x}^{(N_s)} \\
                             \end{bmatrix}
\end{equation}
(implicitly here we treate observations as row vectors)

Thus the sample covariance matrix is 
\begin{equation}
\hat{\Sigma} = \mathbf{X}^\top \mathbf{X}
\end{equation}
Note that $ \hat{\Sigma}_{ij} = \sum X_{ki} X_{kj} = \sum \mathbf{x}^{(k)}_i \mathbf{x}^{(k)}_j$ where $i$ and $j$ denote factors and $k$ indicates a particular sample.

# Linear transformations and covariance
We may consider a linear transformation of a random vector $\mathbf{c}^\top \mathbf{x}$. What is it's covariance matrix
\begin{equation}
\operatorname{cov}(\mathbf{c}^\top \mathbf{x}) = E(\mathbf{c}^\top \mathbf{x}\mathbf{x}^\top\mathbf{c}) = \mathbf{c}^\top E(\mathbf{x}\mathbf{x}^\top) \mathbf{c} = \mathbf{c}^\top \Sigma \mathbf{c}.  
\end{equation}

*Note* that if $\mathbf{c} $ is a vector, $\mathbf{c}^\top \mathbf{x}$ is a scalar, so the covariance is just the variance.


# So, is there a way to formulate dimensionality reduction based on this concept?

If we restrict our consideration to unit vectors $c$, we might seek to find a $c$ which maximizes 
$\mathbf{c}^\top \Sigma \mathbf{c}$ , i.e. which single dimensional variable depending linearly on $\mathbf{x}$ has maximum variance?

Replacing $\Sigma$ with $\hat{\Sigma}$ results in maximizing
\begin{equation}
 \mathbf{c}^\top \mathbf{X}^\top \mathbf{X} \mathbf{c} = \lVert \mathbf{X} \mathbf{c} \rVert^2
 \end{equation}
which is the Rayleigh quotient and attains a maximum of the largest eigenvalue of $\mathbf{X}^\top \mathbf{X}$ when $\mathbf{c}$ is the corresponding eigenvector.

We can show that the subspace spanned by the first k eigenvectors is the k-dimensional subspace that perserves the greatest variability of the data. (Due to the fact that eigenvectors are orthogonal for these matrices).

This leads us to consider the Eigendecomposition of the sample covariance matrix
\begin{equation}
\mathbf{X}^\top \mathbf{X} = \mathbf{V} \Lambda \mathbf{V}^\top
\end{equation}
where $\mathbf{V}$ is an orthogonal matrix and $\Lambda$ is a diagonal matrix of eigenvalues.

This is closely connected to the singular value decomposition of $\mathbf{X}$
\begin{equation}
\mathbf{X} = \mathbf{U} \mathbf{S} \mathbf{V}^\top
\end{equation}
Note that then 
\begin{equation}
\mathbf{X}^\top \mathbf{X} =  \mathbf{V} \mathbf{S} \mathbf{U}^\top  \mathbf{U} \mathbf{S} \mathbf{V}^\top 
\end{equation}
since $\mathbf{U}$ is unitary/orthogonal $\mathbf{U}^\top  \mathbf{U} = \mathbf{I}$ and

\begin{equation}
\mathbf{X}^\top \mathbf{X} =  \mathbf{V} \mathbf{S}^2 \mathbf{V}^\top 
\end{equation}
so the eigenvalues of the sample covariance are the squares of the singular values of $\mathbf{X}$.

# Approximating $\mathbf{X}$ with principal components
This leads us to a view of how principal components are useful for approximating $\mathbf{X}$
we can denote $\mathbf{T} = \mathbf{U} \mathbf{S}$
\begin{equation}
\mathbf{X} = \mathbf{T} \mathbf{V}^\top.
\end{equation}
$\mathbf{T}$ is the score matrix it consists of the magnitude of the projection of each row of $\mathbf{X}$ onto the eigenvectors which are the columns of $\mathbf{V}$; explicitly 
\begin{equation}
T_{ij} = \mathbf{x}^{(i)} \cdot \mathbf{v}_j.
\end{equation}

We then can form an approximation of $\mathbf{X}$ by including only the first k principal components. 

## How do we modify the matrices $\mathbf{T}$ and $\mathbf{V}$ that we have?

# Suggested solution follows

Please

Try

This

At

Home

before 

scrolling

further.

[There's a monster at the end of this notebook](https://www.goodreads.com/book/show/44186.The_Monster_at_the_End_of_this_Book). You don't want to get to the monster do you?
![Image of monster](https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1388193494l/44186.jpg)


## How do we modify the matrices $\mathbf{T}$ and $\mathbf{V}$ that we have?

Truncate to $k$ principal components by using only the first $k$ columns of $\mathbf{T}$ and $\mathbf{V}$

\begin{equation}
\mathbf{X} \approx \mathbf{T}_k \mathbf{V}_k^\top = \sum_{i=0}^k \mathbf{t}_i \mathbf{v}_i^\top = \sum_{i=0}^k \mathbf{t}_i \otimes \mathbf{v}_i
\end{equation}

so we can see that given the values or scores of the principal components and the vectors defining their directions we can reconstruct $\mathbf{X}$ or even a single observation $\mathbf{x}^{(i)}$ from the score matrix $\mathbf{T}$ or just the scores
\begin{equation}
t_j = \mathbf{x}^{(i)} \cdot \mathbf{v}_j
\end{equation}

So given new data items, we could also make use of the matrix $\mathbf{V}$ to determine the represenation of $\mathbf{x}^{(i)}$ according the principal components.

# An algorithm

In [ ]:
def PCA_algorithm_evd(X):
    """
    An algorithm to find the principal components by Eigendecomposition of X' * X
    Args:
        X (ndarray): A n_s (number of samples) by n_d (number of features/dimensions)
            matrix where each row is a sample and each column corresponds to features,
            where the columns have zero mean.
    Returns:
        V (ndarray): A n_d by n_d matrix where the columns represents the ordered 
            directions that preserve maximal variance of the data.
        ev_sorted (ndarray): a n_d vector of the squared eigenvalues of X' * X such 
            that the k-th value is proportional to the amount of variance preserved
            when the data is projected onto the subspace spanned by V[:,k]
        scores (ndarray): a n_s by n_d matrix where the rows are the values of the 
            samples in the coordinate system defined by V.
    """
    covX = X.T @ X/(X.shape[0]-1)
    ev, v = np.linalg.eig(covX)
    ev_sort_idx = np.argsort(-np.abs(ev))
    ev_sorted = ev[ev_sort_idx]
    v_sorted = v[:,ev_sort_idx]
    scores = X @ v_sorted
    return v_sorted, scores, ev_sorted

# How does this work for our sample data

In [ ]:
X = data_trans.T
V, T, explained_variance = PCA_algorithm_evd(X)
total_variance = np.sum(explained_variance)
explained_variance = explained_variance/total_variance

In [ ]:
explained_variance

In [ ]:
V

In [ ]:
plt.figure("2D-example")
#plt.plot(data[0],data[1],'o')
plt.plot(data_trans[0],data_trans[1],'o')
plt.quiver(0, 0, V[0,0], V[1,0],  angles='xy', scale_units='xy', scale=1, zorder=1e6)
plt.quiver(0, 0, V[0,1], V[1,1],  angles='xy', scale_units='xy', scale=1, zorder=1e6)
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")

In [ ]:
plt.figure("2D-loading")
l1 = explained_variance[0]
l2 = explained_variance[0]
#plt.plot(data[0],data[1],'o')
plt.plot(T[:,0],T[:,1],'o')
plt.xlabel("$t_1$")
plt.ylabel("$t_2$")

In [ ]:
plt.figure("2D-check")
plt.plot(data_trans[0],data_trans[1],'o')
plt.plot(T[:,0], T[:,1],'o')
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")

# Okay, let's do this for something a bit more _realistic_

Motivation, for reduced order modelling of fractional flow reserve (FFR), we would like to make use of the full geometrical data available, but if we include all features we have way more input features than samples. Thus it will be ill posed to quantify the contribution of each feature. PCA can help reduce the full set of features to a lower dimensional set of features such that thevariation of the data is represented as input, but one doesn't need to know the contribution of each particular input.

Here we start with a set of synthetic axi-symmetric geometries. Let's do PCA to see what we can find out.



In [ ]:
r_data = []
num_samples_per_cls = 5
offset = 1.5
r0 = np.ones(10)
x = np.linspace(0,1,10)
np.random.seed()
plt.figure()
for k in range(num_samples_per_cls):
    r1 = np.cos(2*np.pi*k*x)  + offset
    r_data.append(r1)
    plt.plot(x, r1)
    
plt.figure()
for k in range(num_samples_per_cls):
    r1 = -1 + 4*np.random.rand()*abs(x - 0.3 - 0.5*np.random.rand())  + offset
    r_data.append(r1)
    plt.plot(x, r1)

plt.figure()
for k in range(num_samples_per_cls):
    r1 = 1 - 2*np.random.rand()*(x >0.25*np.random.rand())*( x < 0.75) + offset
    r_data.append(r1)
    plt.plot(x, r1) 
    
r_data = np.array(r_data) + 1.5

# Let's try PCA on this data set. Try the following exercises.

* Compute the PCA scores for this data.

* Can you determine how many components are needed to account for most of the variability (80%, 90%, etc?) in the data?

* For a given sample/observation set how well do 1, 2, 3, ... principal components approximate the full feature set?

* For a given sample observation what PC is it most like? 

* How well does this single PC approximate the observation?


In [ ]:
PCA_algorithm_evd?
# Start implementing your solution here...

# Suggested solution follows

Please

Try

This

At

Home

before 

scrolling

further.

[There's a monster at the end of this notebook](https://www.goodreads.com/book/show/44186.The_Monster_at_the_End_of_this_Book). You don't want to get to the monster do you?
![Image of monster](https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1388193494l/44186.jpg)



In [ ]:
r_data
X_all = r_data - np.mean(r_data, axis=0) #PCA assumes 0-mean data
# Compute PCA representation
train_idx = np.ones(len(X_all))>0
X = X_all[train_idx]
v_sorted,  scores, eigenvals = PCA_algorithm_evd(X)
total_var = np.sum(eigenvals)
var_exp = eigenvals/total_var


plt.figure()
plt.plot(var_exp)
plt.ylabel("Var explained by PC_i")
plt.xlabel("PC index")


plt.figure()
plt.plot(np.cumsum(var_exp))
plt.ylabel("Cumulative variance explained by all PC_0 .. PC_i")
plt.xlabel("PC index")

d_idx = 3 # data observation to look at
k = 5 # number of PC to include in reconstruction
plt.figure()
pc = np.argmax(scores[d_idx])
plt.plot(r_data[d_idx] -  np.mean(r_data, axis=0), color="black", label='data') # shift data to be centered for comparison with PCs
plt.plot(v_sorted[:,0:k] @ scores[d_idx,0:k], label="k-PC approx") # reconstruct with all k PCs
plt.plot(scores[d_idx][pc]*v_sorted[:,pc], '--', label='Best PC') # use only the closest PC
plt.legend()

In [ ]:
# Check all cases
for d_idx, d in enumerate(r_data):
    plt.figure()
    pc = np.argmax(scores[d_idx])
    plt.plot(r_data[d_idx] -  np.mean(r_data, axis=0), color="black", label='data') # shift data to be centered for comparison with PCs
    plt.plot(v_sorted[:,0:k] @ scores[d_idx,0:k], label="k-PC approx") # reconstruct with all k PCs
    plt.plot(scores[d_idx][pc]*v_sorted[:,pc], '--', label='Best PC') # use only the closest PC
    plt.legend()

# Further topics

* PCA Regression regress onto PC instead of the data itself then compute regression coefficients of X from those of T to avoid overfitting issues.

# PCA regression with NHANES data

Below you find some code to load the [NHANES](https://wwwn.cdc.gov/nchs/nhanes) 2015-2016 data into a [pandas](https://pandas.pydata.org/pandas-docs/stable/?v=20191001113306) data frame. Subsequently we will compare linear regression of blood pressure onto other variables in the data set.

In [ ]:
import pandas as pd

In [ ]:
df_bmx = pd.read_sas("BMX_I.XPT")
df_bpx = pd.read_sas("BPX_I.XPT")
df = df_bmx.join(df_bpx, lsuffix="BMX", rsuffix="BPX")

# NHANES
Here is an overview of the variable names
https://wwwn.cdc.gov/nchs/nhanes/Search/variablelist.aspx?Component=Examination&CycleBeginYear=2015

We will use only a few of these and limit ourselves to the continuous variables.

In [ ]:
response_vars = """BPXDI2
BPXPLS
BPXSY2""".split("\n")

In [ ]:
predictor_vars = """BPXCHR
BMDAVSAD
BMIWT
BMXARMC
BMXARML
BMXBMI
BMXHT
BMXLEG
BMXRECUM
BMXSAD1
BMXSAD2
BMXSAD3
BMXSAD4
BMXWAIST
BMXWT""".split("\n")

In [ ]:
df.fillna(df.median(), inplace=True)
df_combined = df[response_vars+predictor_vars]
df_responses = df[response_vars]
df_predictors = df[predictor_vars]

### Convert data frame to raw data
The code that follows makes use of the `sklearn` and `statsmodels` integration with `pandas` dataframes for ease of use. Thus instead of the PCA code presented above we use `sklearn` to compute the PCA of the data.
To work directly on the data as a matrix, we can access `df.values` which is a numpy array.


In [ ]:
df_predictors.values

In [ ]:
df_combined.describe()

In [ ]:
df_responses.describe()

In [ ]:
df_combined.plot(x="BMXBMI", y="BPXPLS", kind="scatter", alpha=0.25)

In [ ]:
import statsmodels.api as sm

In [ ]:
res = sm.OLS(df_responses["BPXPLS"], sm.add_constant(df_predictors, prepend=False)).fit()
print('OLS on original data')
print(res.params)
print(res.aic)
print(res.rsquared)
print(res.summary())

# We can try the same with PCA components

In [ ]:
import sklearn.decomposition
pca_obj = sklearn.decomposition.PCA()
pca_obj.fit(df_predictors)

In [ ]:
PCA_scores = pca_obj.transform(df_predictors)

In [ ]:
res = sm.OLS(df_responses["BPXPLS"], sm.add_constant(PCA_scores, prepend=False)).fit()
print('OLS on PCA data')
print(res.params)
print(res.aic)
print(res.rsquared)
print(res.summary())

In [ ]:
plt.figure()
plt.plot(pca_obj.explained_variance_ratio_)